# Z-Image (Lumina-2) : Generation Avancee avec ComfyUI

**Module :** 02-Images-Advanced
**Niveau :** Avance
**Duree estimee :** 30 minutes
**Statut :** FONCTIONNEL

## Introduction

Z-Image utilise l'architecture **Lumina-Next-SFT** via le wrapper Diffusers pour la generation d'images haute qualite. Ce notebook utilise le node `LuminaDiffusersNode` qui integre le pipeline HuggingFace Diffusers directement dans ComfyUI.

### Architecture

| Composant | Details |
|-----------|---------|
| **Pipeline** | LuminaPipeline (diffusers 0.34+) |
| **Modele** | Alpha-VLLM/Lumina-Next-SFT-diffusers (~10GB) |
| **VAE** | SDXL VAE (sdxl_vae.safetensors) |
| **Resolution** | 1024x1024 (natif) |

### Note sur l'approche GGUF

L'approche GGUF (z_image_turbo + gemma CLIP) a ete abandonnee en raison d'incompatibilites dimensionnelles (2560 vs 2304 entre RecurrentGemma et Gemma-2).

## Prerequis
- Service `comfyui-qwen` actif
- Node installe : LuminaDiffusersNode (ComfyUI-Lumina-Next-SFT-DiffusersWrapper)
- VAE : sdxl_vae.safetensors dans models/vae/

In [ ]:
# 1. Configuration de l'environnement
import os
import requests
import json
import time
from PIL import Image
from io import BytesIO
from dotenv import load_dotenv
import matplotlib.pyplot as plt

# Chargement des variables d'environnement
load_dotenv("../00-GenAI-Environment/.env")

COMFYUI_URL = "http://localhost:8188"
COMFYUI_TOKEN = os.getenv("COMFYUI_AUTH_TOKEN")

if not COMFYUI_TOKEN:
    raise ValueError("❌ Token COMFYUI_AUTH_TOKEN manquant dans le fichier .env")

HEADERS = {"Authorization": f"Bearer {COMFYUI_TOKEN}"}

print("✅ Configuration chargée")
print(f"Target URL: {COMFYUI_URL}")

In [ ]:
# 2. Definition du Workflow Z-Image (Diffusers)
# Ce workflow utilise LuminaDiffusersNode qui telecharge automatiquement
# le modele depuis HuggingFace (Alpha-VLLM/Lumina-Next-SFT-diffusers)

def create_zimage_workflow(prompt: str, 
                           negative_prompt: str = "blurry, low quality, text, watermark",
                           width: int = 1024, 
                           height: int = 1024,
                           steps: int = 30,
                           guidance: float = 4.0,
                           seed: int = None) -> dict:
    """
    Cree un workflow Z-Image avec le wrapper Diffusers.
    
    Args:
        prompt: Description de l'image a generer
        negative_prompt: Elements a eviter
        width/height: Dimensions (1024x1024 recommande)
        steps: Etapes de diffusion (20-40 optimal)
        guidance: Guidance scale (3-5 optimal pour Lumina)
        seed: Graine pour reproductibilite (-1 = aleatoire)
    
    Returns:
        Workflow JSON pret pour ComfyUI
    """
    if seed is None:
        seed = -1  # -1 = random dans LuminaDiffusersNode
    
    return {
        # LuminaDiffusersNode - Le coeur de la generation
        # Note: Le node gere en interne la resolution via le scheduler
        "1": {
            "class_type": "LuminaDiffusersNode",
            "inputs": {
                "model_path": "Alpha-VLLM/Lumina-Next-SFT-diffusers",
                "prompt": prompt,
                "negative_prompt": negative_prompt,
                "num_inference_steps": steps,
                "guidance_scale": guidance,
                "seed": seed,
                "batch_size": 1,
                "scaling_watershed": 0.3,
                "proportional_attn": True,
                "clean_caption": True,
                "max_sequence_length": 256,
                "use_time_shift": False,
                "t_shift": 4,
                "strength": 1.0
            }
        },
        # VAELoader - SDXL VAE pour le decodage
        "2": {
            "class_type": "VAELoader",
            "inputs": {
                "vae_name": "sdxl_vae.safetensors"
            }
        },
        # VAEDecode - Conversion latent vers image
        "3": {
            "class_type": "VAEDecode",
            "inputs": {
                "samples": ["1", 0],
                "vae": ["2", 0]
            }
        },
        # SaveImage - Sauvegarde du resultat
        "4": {
            "class_type": "SaveImage",
            "inputs": {
                "filename_prefix": "Z-Image-Lumina",
                "images": ["3", 0]
            }
        }
    }

# Workflow de test basique
z_image_workflow = create_zimage_workflow(
    prompt="A vibrant sunset over a mountain lake, reflection in water, photorealistic",
    steps=20,
    guidance=4.0,
    seed=42
)

print("Workflow Z-Image (Diffusers) defini")
print("  - Node principal: LuminaDiffusersNode")
print("  - Modele: Alpha-VLLM/Lumina-Next-SFT-diffusers")
print("  - VAE: SDXL")
print("  - Output node: '4'")

In [ ]:
# 3. Fonction de Generation Helper
def generate_z_image(prompt, seed=None, steps=20, guidance=4.0, width=1024, height=1024):
    """
    Genere une image avec Z-Image (Lumina Diffusers).
    
    Args:
        prompt: Description de l'image
        seed: Graine pour reproductibilite (None/-1 = aleatoire)
        steps: Nombre d'etapes (20-40 recommande)
        guidance: Guidance scale (3-5 recommande)
        width/height: Dimensions
    
    Returns:
        PIL.Image ou None en cas d'erreur
    """
    if seed is None:
        seed = -1
    
    # Creer le workflow
    workflow = create_zimage_workflow(
        prompt=prompt,
        seed=seed,
        steps=steps,
        guidance=guidance,
        width=width,
        height=height
    )
    
    payload = {"prompt": workflow}
    
    print(f"Envoi de la requete... (Seed: {seed})")
    resp = requests.post(f"{COMFYUI_URL}/prompt", json=payload, headers=HEADERS)
    
    if resp.status_code != 200:
        print(f"Erreur API: {resp.text}")
        return None
        
    prompt_id = resp.json()["prompt_id"]
    print(f"Tache ID: {prompt_id}")
    print("Generation en cours (premiere execution peut prendre plusieurs minutes pour telecharger le modele)...")
    
    # Polling avec timeout etendu (premier run telecharge ~10GB)
    max_wait = 600  # 10 minutes pour le premier run
    start_time = time.time()
    
    while time.time() - start_time < max_wait:
        history_resp = requests.get(f"{COMFYUI_URL}/history/{prompt_id}", headers=HEADERS)
        if history_resp.status_code == 200:
            history_data = history_resp.json()
            if prompt_id in history_data:
                prompt_data = history_data[prompt_id]
                status = prompt_data.get('status', {})
                
                if status.get('completed'):
                    elapsed = time.time() - start_time
                    print(f"Generation terminee en {elapsed:.1f}s")
                    
                    # Recuperation image (node 4 = SaveImage)
                    outputs = prompt_data.get('outputs', {})
                    if '4' in outputs and 'images' in outputs['4']:
                        output_data = outputs['4']['images'][0]
                        filename = output_data['filename']
                        subfolder = output_data.get('subfolder', '')
                        img_type = output_data.get('type', 'output')
                        
                        img_url = f"{COMFYUI_URL}/view?filename={filename}&subfolder={subfolder}&type={img_type}"
                        img_resp = requests.get(img_url, headers=HEADERS)
                        
                        if img_resp.status_code == 200:
                            return Image.open(BytesIO(img_resp.content))
                    
                    print("Pas d'image dans les outputs")
                    return None
                    
                if status.get('status_str') == 'error':
                    print(f"Erreur: {status.get('messages', 'Unknown')}")
                    return None
        
        time.sleep(2)
    
    print(f"Timeout apres {max_wait}s")
    return None

In [ ]:
# 4. Test de Generation
# Note: La premiere execution telecharge le modele (~10GB), prevoir 5-10 minutes

prompt = "Cinematic photography of a samurai robot in a neon cyberpunk city, raining, reflections, 8k, highly detailed"

print("Lancement de la generation Z-Image (Lumina Diffusers)...")
print(f"Prompt: {prompt}")
print("\nNote: Le premier lancement telecharge le modele (~10GB)")

image = generate_z_image(prompt, seed=42, steps=20, guidance=4.0)

if image:
    plt.figure(figsize=(10, 10))
    plt.imshow(image)
    plt.title(f"Z-Image: {prompt[:50]}...", fontsize=10)
    plt.axis("off")
    plt.tight_layout()
    plt.show()
    
    print(f"\nImage generee: {image.size[0]}x{image.size[1]}")
else:
    print("\nEchec de la generation. Verifiez:")
    print("  1. Le service ComfyUI est actif")
    print("  2. Le node LuminaDiffusersNode est installe")
    print("  3. Le VAE SDXL est present dans models/vae/")

## Exercices

### Exercice 1: Exploration des parametres
Testez differentes valeurs de guidance (2.0, 4.0, 6.0) avec le meme prompt et seed pour observer l'impact sur la fidelite au prompt.

### Exercice 2: Comparaison de styles
Generez des images avec des styles differents:
- "A peaceful mountain lake at sunrise, photorealistic"
- "A peaceful mountain lake at sunrise, watercolor painting style"
- "A peaceful mountain lake at sunrise, anime style"

### Exercice 3: Resolution
Testez differentes resolutions (512x512, 768x768, 1024x1024) et observez l'impact sur la qualite et le temps de generation.

## Notes Techniques

### Parametres recommandes
| Parametre | Plage | Recommande | Impact |
|-----------|-------|------------|--------|
| steps | 15-50 | 20-30 | Qualite vs vitesse |
| guidance | 2-7 | 3-5 | Fidelite au prompt |
| scaling_watershed | 0.0-1.0 | 0.3 | Reduction saturation |

### Differences avec Qwen
- **Lumina/Z-Image**: Generation text-to-image de haute qualite, meilleur pour scenes complexes
- **Qwen**: Specialise dans l'edition d'images existantes, image-to-image
- Lumina utilise un VAE 4-channel SDXL, Qwen utilise un VAE 16-channel

### Premier lancement
Le premier lancement telecharge automatiquement le modele (~10GB) depuis HuggingFace.
Cela peut prendre 5-10 minutes selon votre connexion.

### Fix technique (Janvier 2025)
Le node `LuminaDiffusersNode` a ete mis a jour pour utiliser `LuminaPipeline` au lieu de
`LuminaText2ImgPipeline` (renomme dans diffusers 0.34+).